In [1]:
import pandas as pd
import csv
import json
import mysql.connector
from mysql.connector import errorcode

from dotenv import load_dotenv
import os

In [2]:
# Una vez ya tenemos los datos, ahora necesitamos el puente a MySQL 
# Creamos las funciones que necesita nuestro puente

# Recordar que tenemos que tener hecho pip install python-dotenv
# Y en esta misma carpeta en la que ejecutáis, tener un archivo .env 
# que dentro ponga MYSQL_PASSWORD = 'aquí ponéis vuestra contraseña'

load_dotenv()
password = os.getenv("MYSQL_PASSWORD")

# Función conectar
load_dotenv()

def conectar():
    try:
        cnx = mysql.connector.connect(
            user='root',
            password=os.getenv("MYSQL_PASSWORD"),
            host='127.0.0.1',
            auth_plugin='mysql_native_password')
        
        print('Llamada (( ☎ )) a SQL\n')

    except mysql.connector.Error as err:
        print(f'Error: {err}')

    return cnx

# Llama a la función
cnx = conectar()
cursor = cnx.cursor()
print('SQL ha contestado (( ☎ )) está listo para trabajar\n')


Llamada (( ☎ )) a SQL

SQL ha contestado (( ☎ )) está listo para trabajar



In [ ]:
def crear_base_de_datos_y_tablas():
    cnx = conectar()
    if cnx is None:
        return

    cursor = cnx.cursor()

    try:
        # Creamos la base de datos
        cursor.execute("CREATE DATABASE IF NOT EXISTS MusicStreamStupendo")
        cursor.execute("USE MusicStreamStupendo")

        # Tabla artistas 
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS artistas (
            id_artista VARCHAR(45) PRIMARY KEY,
            nombre VARCHAR(100),
            biografia TEXT,
            reproducciones INT,
            escuchas INT,
            artistas_similares TEXT
        )
        """)

        # Tabla generos
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS generos (
            id_genero INT AUTO_INCREMENT PRIMARY KEY,
            nombre VARCHAR(50)
        )
        """)

        # Tabla albumes
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS albumes (
            id_album VARCHAR(50) PRIMARY KEY,
            nombre VARCHAR(100),
            año_lanzamiento DATE,
            popularidad INT,
            id_artista VARCHAR(45),
            FOREIGN KEY (id_artista) REFERENCES artistas(id_artista)
        )
        """)

        # Tabla top_tracks
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS top_tracks (
            id_cancion INT PRIMARY KEY,
            titulo VARCHAR(100),
            año_lanzamiento VARCHAR(45),
            id_genero INT,
            id_artista VARCHAR(45),
            FOREIGN KEY (id_genero) REFERENCES generos(id_genero),
            FOREIGN KEY (id_artista) REFERENCES artistas(id_artista)
        )
        """)

        # Intertabla géneros
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS artistas_generos (
            id_artista VARCHAR(45),
            id_genero INT,
            PRIMARY KEY (id_artista, id_genero),
            FOREIGN KEY (id_artista) REFERENCES artistas(id_artista),
            FOREIGN KEY (id_genero) REFERENCES generos(id_genero)
        )
        """)

        # Intertabla albumes_canciones
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS albumes_canciones (
            id_album VARCHAR(50),
            id_cancion INT,
            PRIMARY KEY (id_album, id_cancion),
            FOREIGN KEY (id_album) REFERENCES albumes(id_album),
            FOREIGN KEY (id_cancion) REFERENCES top_tracks(id_cancion)
        )
        """)

        cnx.commit()
        print('> Base de datos y tablas creadas correctamente.')
    except mysql.connector.Error as err:
        print(f'Error al crear tablas: {err}')

    finally:
        cursor.close()
        cnx.close()

# Llamar a la función para crear la base de datos y tablas
crear_base_de_datos_y_tablas()


Llamada (( ☎ )) a SQL

> Base de datos y tablas creadas correctamente.


In [ ]:
# Vamos a empujar a los artistas con el .csv con ID de los resultados de last.fm
# Hay campos que SQL va a dejar en tierra y no los va a embarcar porque no identifica 'NaN'


artistas = pd.read_csv('../csv/total_artistas_lastfm_id.csv') #revisamos siempre que la ruta relativa es correcta

for columna in artistas.itertuples(index=False):
    artista, playcount, listeners, biografia_resumen, similares, artista_id  = columna

    try:
        cursor.execute("USE MusicStreamStupendo")
        sql = """INSERT INTO artistas (nombre, reproducciones, escuchas, biografia, artistas_similares, id_artista)
                VALUES (%s, %s, %s, %s, %s, %s)"""
        cursor.execute(sql, (artista, playcount, listeners, biografia_resumen, similares, artista_id))
        print('Tout va bien: sin incidencias.')
    except Exception as e:
        cnx.rollback()
        print('Rien de rien:', e)
    else:
        cnx.commit()

Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Rien de rien: 1054 (42S22): Unknown column 'nan' in 'field list'
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Rien de rien: 1054 (42S22): Unknown column 'nan' in 'field list'
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Tout va bien: sin incidencias.
Rien de rien: 1054 (42S22): Unknown column 'nan' in 'field list'